In [ ]:
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms

In [ ]:
class ImageDataset(Dataset):
    def __init__(self, img_paths, seg_paths, transform=None):
        self.img_paths = img_paths
        self.seg_paths = seg_paths
        self.transform = transform

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        img = cv2.imread(self.img_paths[idx])
        seg = cv2.imread(self.seg_paths[idx])
        if self.transform:
            img, seg = self.transform(img, seg)
        return torch.tensor(img, dtype=torch.float).permute(2, 0, 1) / 128.0 - 1.0, torch.tensor(seg, dtype=torch.float).permute(2, 0, 1) / 128.0 - 1.0


In [ ]:
class Transform:
  def __init__(self):
        self.angles = range(-2, 3)
        
  def generate_shifts(range_x, range_y):
    shifts = []
    for x in range(-range_x, range_x + 1):
        for y in range(-range_y, range_y + 1):
            shifts.append([x, y])
    return shifts

  def __call__(self, img, seg):
        augmented_imgs = []
        augmented_segs = []
        for angle in self.angles:
            for shift in self.shifts:
                M = cv2.getRotationMatrix2D((128/2,128/2), angle, 1)
                shiftM = np.float32([[1,0,shift[0]],[0,1,shift[1]]])
                rotated_img = cv2.warpAffine(img, M, (128, 128))
                rotated_seg = cv2.warpAffine(seg, M, (128, 128))
                shifted_img = cv2.warpAffine(rotated_img, shiftM, (128, 128))
                shifted_seg = cv2.warpAffine(rotated_seg, shiftM, (128, 128))
                augmented_imgs.append(shifted_img)
                augmented_segs.append(shifted_seg)
        return np.stack(augmented_imgs), np.stack(augmented_segs)
  

In [ ]:
class UNet(nn.Module):
    def __init__(self):
        super(UNet, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 3, kernel_size=3, padding=1),
            nn.Tanh()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [ ]:
path_x = 'Data/X/'
path_y = 'Data/Y2/'
img_paths = sorted([os.path.join(path_x, f) for f in os.listdir(path_x) if f.endswith('.jpg')])[:-5]
seg_paths = sorted([os.path.join(path_y, f) for f in os.listdir(path_y) if f.endswith('.jpg')])[:-5]
transform = Transform()
dataset = ImageDataset(img_paths, seg_paths, transform=transform)
dataloader = DataLoader(dataset, batch_size=30, shuffle=True, num_workers=4)

# Initialize model, loss, and optimizer
model = UNet()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [ ]:
model.train()
for epoch in range(10):
    for imgs, segs in dataloader:
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, segs)
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

model.eval()
test_files = ["A-train0101.jpg", "A-train0102.jpg", "A-train0103.jpg", "A-train0104.jpg", "A-train0105.jpg"]
X_test = [cv2.imread(os.path.join(path_x, f)) for f in test_files]
X_test = [torch.tensor(img, dtype=torch.float).permute(2, 0, 1) / 128.0 - 1.0 for img in X_test]
X_test = torch.stack(X_test)
with torch.no_grad():
    y_out = model(X_test)
y_out = y_out * 128.0 + 128.0
for i, img in enumerate(y_out):
    img = img.permute(1, 2, 0).numpy().astype(np.uint8)
    cv2.imwrite(f'y{i}.jpg', img)

torch.save(model.state_dict(), 'dev.pth')

In [ ]:
import os
import cv2
import numpy as np
import torch
from torchvision import transforms

preprocess = transforms.Compose([
    transforms.ToTensor(),          
    transforms.Normalize((128, 128, 128), (128, 128, 128))  
])

model = UNet()  
model.load_state_dict(torch.load('dev.pth'))
model.eval()

test_folder = "devtest/"
output_folder = "devoutput/"
test_files = os.listdir(test_folder)

images = []
for file_name in test_files:
    img_path = os.path.join(test_folder, file_name)
    img = cv2.imread(img_path)
    img = preprocess(img) 
    images.append(img)

X_test = torch.stack(images)

with torch.no_grad():
    y_out = model(X_test)

for i, img_tensor in enumerate(y_out):
    img = img_tensor.permute(1, 2, 0).numpy()
    img = (img * 128.0 + 128.0).astype(np.uint8)  
    output_path = os.path.join(output_folder, test_files[i])
    cv2.imwrite(output_path, img)